In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Set the path to the dataset
dataset_path = r'D:\IIT\Subjects\(4605)IRP\Devlo\Augmented_DataSet'

In [3]:
# Define image dimensions and other constants
img_height, img_width = 256, 256  # Image resolution
batch_size = 32
num_classes = 4  # 4 classes: Healthy, Live Wood, Pink Wax, Stem Canker

In [4]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 80% training, 20% validation
)

In [5]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training subset
)

Found 7656 images belonging to 4 classes.


In [6]:
# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation subset
)

Found 1914 images belonging to 4 classes.


In [7]:
# Build CNN model
model = Sequential()

In [8]:
# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

c:\Users\ridmi\.conda\envs\TenserFlow\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Flatten and Fully Connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with 4 classes

In [10]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,492 (56.61 MB)

 Trainable params: 14,839,492 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20  # Adjust the number of epochs as needed
)

c:\Users\ridmi\.conda\envs\TenserFlow\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3930 - loss: 1.3621

c:\Users\ridmi\.conda\envs\TenserFlow\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


239/239 ━━━━━━━━━━━━━━━━━━━━ 283s 1s/step - accuracy: 0.3933 - loss: 1.3614 - val_accuracy: 0.5704 - val_loss: 1.0208
Epoch 2/20
  1/239 ━━━━━━━━━━━━━━━━━━━━ 3:20 844ms/step - accuracy: 0.4688 - loss: 1.0958

c:\Users\ridmi\.conda\envs\TenserFlow\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


239/239 ━━━━━━━━━━━━━━━━━━━━ 35s 142ms/step - accuracy: 0.4688 - loss: 1.0958 - val_accuracy: 0.5678 - val_loss: 1.0030
Epoch 3/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 277s 1s/step - accuracy: 0.5606 - loss: 1.0111 - val_accuracy: 0.5323 - val_loss: 1.0117
Epoch 4/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 34s 139ms/step - accuracy: 0.5938 - loss: 1.1791 - val_accuracy: 0.5588 - val_loss: 0.9456
Epoch 5/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 276s 1s/step - accuracy: 0.5939 - loss: 0.9624 - val_accuracy: 0.6028 - val_loss: 0.8831
Epoch 6/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 35s 142ms/step - accuracy: 0.5000 - loss: 1.0176 - val_accuracy: 0.6006 - val_loss: 0.8809
Epoch 7/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 279s 1s/step - accuracy: 0.6132 - loss: 0.9395 - val_accuracy: 0.6107 - val_loss: 0.8968
Epoch 8/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 34s 137ms/step - accuracy: 0.5625 - loss: 1.1378 - val_accuracy: 0.6043 - val_loss: 0.8918
Epoch 9/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 279s 1s/step - accuracy: 0.6377 - loss: 0.8484 - val_accurac

In [ ]:
# Save the model
model.save('D:\IIT\Subjects\(4605)IRP\Devlo\models\CNN_model.h5')

In [14]:
# Evaluate the model
score = model.evaluate(validation_generator)
print(f"Validation Loss: {score[0]}")
print(f"Validation Accuracy: {score[1]}")

60/60 ━━━━━━━━━━━━━━━━━━━━ 35s 583ms/step - accuracy: 0.6794 - loss: 0.8204
Validation Loss: 0.8336114883422852
Validation Accuracy: 0.6781609058380127
